In [14]:
# STANDALONE TO DOWNLOAD MOVIES GIVEN CSV FILE

import csv
import time
import os
import requests
import numpy as np
import pandas as pd
import urllib
import pickle

#########################################################

# Download one image to filepath, note that imgpath is prepended with a /
def _downloadImageToFile(imgpath, filename):
    # Does not return anything
    urllib.urlretrieve("https://image.tmdb.org/t/p/w500" + imgpath, filename)
    
def downloadImagesToFolder(csv_file, foldername): 
    # Folder name without any trailing /; If folder does not exist, create it
    if not os.path.exists(foldername):
        os.makedirs(foldername)

    df = pd.read_csv(csv_file)
    df = np.array(df[['tmdb_id', 'poster_path']])
    print'total number of posters: ', df.shape[0]
    
    # TMDB limits 4 requests per second
    hit = 3 # Once hit reaches 0, call timer and reset hit to 3
    
    for count, row in df.iterrows():
        mid = str(row['tmdb_id'])
        poster_path = row['poster_path']
        if (type(poster_path) != str) or (poster_path.strip() == ''):
            continue
        _downloadImageToFile(imgpath=poster_path, filename=foldername + "/" + mid + ".jpg")
        hit -= 1
        if hit <= 0:
            hit = 3
            time.sleep(1)
        if count % 200 == 0:
            print'Done downloading %d posters' % count

In [15]:
downloadImagesToFolder(csv_file = '../data/movie-posters-list.csv', foldername='../posters')

('total number of posters: ', 6354)
Done downloading 200 posters
Done downloading 400 posters
Done downloading 600 posters
Done downloading 800 posters
Done downloading 1000 posters
Done downloading 1200 posters
Done downloading 1400 posters
Done downloading 1600 posters
Done downloading 1800 posters
Done downloading 2000 posters
Done downloading 2200 posters
Done downloading 2400 posters
Done downloading 2600 posters
Done downloading 2800 posters
Done downloading 3000 posters
Done downloading 3200 posters
Done downloading 3400 posters
Done downloading 3600 posters
Done downloading 3800 posters
Done downloading 4000 posters
Done downloading 4200 posters
Done downloading 4400 posters
Done downloading 4600 posters
Done downloading 4800 posters
Done downloading 5000 posters
Done downloading 5200 posters
Done downloading 5400 posters
Done downloading 5600 posters
Done downloading 5800 posters
Done downloading 6000 posters
Done downloading 6200 posters
